## First Part (Generating the first DataFrame)

In [1]:
! pip install beautifulsoup4
from bs4 import BeautifulSoup

In [2]:
import requests
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.text)
My_table = soup.find('table',{'class':'wikitable sortable'})

#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
ths=My_table.findAll('th')
headings = [th.text.strip() for th in ths]
headings

['Postcode', 'Borough', 'Neighbourhood']

#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
import pandas as pd
pc=[]
bo=[]
nh=[]

for tr in My_table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    val = [td.text.strip() for td in tds[:3]]
    pc.append(val[0])
    bo.append(val[1])
    nh.append(val[2])
    
df=pd.DataFrame({ headings[0]:pc, headings[1]:bo, headings[2]:nh })
df=df[df['Borough'] !='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, [...]

In [5]:
joined_data=[]
for x in df['Postcode']:
    str= df[df['Postcode']==x]['Neighbourhood'].str.cat(sep=',')
    joined_data.append(str)
df['Neighbourhood']=joined_data 
df=df.drop_duplicates(subset='Postcode')
df.head()    

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Not assigned


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. [...]

In [6]:
t=df[df['Neighbourhood'] =='Not assigned']
t['Neighbourhood']=t['Borough']
df[df['Neighbourhood'] =='Not assigned']=t
df.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queen's Park


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
"number of rows = %i" % df.shape[0]

'number of rows = 103'